In [6]:
import cv2
import joblib
import numpy as np

In [12]:
def load_bounding_boxes(pickle_file):
    return joblib.load(pickle_file)

def apply_mask(frame, bboxes):
    mask = np.zeros_like(frame)
    for bbox in bboxes:
        x, y, w, h = map(int, bbox)
        mask[y:y+h, x:x+w] = frame[y:y+h, x:x+w]
    return mask

def process_video(video_path, bbox_data, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    frame_id = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_key = f'{frame_id:06d}.jpg'
        if frame_key in bbox_data:
            frame = apply_mask(frame, bbox_data[frame_key]['gt_bbox'])
        
        out.write(frame)
        frame_id += 1

    cap.release()
    out.release()

In [21]:
bbox_data = load_bounding_boxes('../../PHALP/bbox_data.pkl')
process_video('./noBbox.mp4', bbox_data, 'masked_output_video.avi')

In [24]:
import cv2

def combine_videos_horizontally(video_path1, video_path2, output_path):
    # Open both video files
    cap1 = cv2.VideoCapture(video_path1)
    cap2 = cv2.VideoCapture(video_path2)

    if not cap1.isOpened() or not cap2.isOpened():
        print("Error opening one of the video files")
        return

    # Get properties of the videos
    width1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
    height1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps1 = cap1.get(cv2.CAP_PROP_FPS)

    width2 = int(cap2.get(cv2.CAP_PROP_FRAME_WIDTH))
    height2 = int(cap2.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps2 = cap2.get(cv2.CAP_PROP_FPS)

    # Determine output video properties
    output_fps = min(fps1, fps2)
    output_height = max(height1, height2)
    aspect_ratio1 = width1 / height1
    aspect_ratio2 = width2 / height2

    # Calculate new widths while preserving aspect ratio
    new_width1 = int(output_height * aspect_ratio1)
    new_width2 = int(output_height * aspect_ratio2)
    output_width = new_width1 + new_width2

    # Create VideoWriter object with higher bitrate
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, output_fps, (output_width, output_height), True)

    while True:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        # Check if there's a frame to read from both videos
        if not ret1 or not ret2:
            break

        # Resize frames to preserve aspect ratio
        frame1 = cv2.resize(frame1, (new_width1, output_height))
        frame2 = cv2.resize(frame2, (new_width2, output_height))

        # Combine the frames horizontally
        combined_frame = cv2.hconcat([frame1, frame2])

        # Write the combined frame to the output video
        out.write(combined_frame)

    # Release resources
    cap1.release()
    cap2.release()
    out.release()
    print(f"Combined video saved to {output_path}")


# Example usage
video_path1 = '/Users/jackgengqc/Downloads/4DHumans.ipynb video.mp4'
video_path2 = '/Users/jackgengqc/Downloads/blackout.mp4'
output_video = 'combined_video.avi'
combine_videos_horizontally(video_path1, video_path2, output_video)


Combined video saved to combined_video.avi
